# Threading

- Para poder ejecutar multiples **flujos de instrucciones** a la vez.

## Procesos y núcleos

1. **Proceso**: Programa en ejecucion que ocupa espacio en la RAM del PC. Se sigue un flujo de instrucciones.

2. **Nucleos**: La CPU ejecuta instrucciones en un nucleo, es el que hace la aritmetica y pasa linea a linea

- CPUs modernas tienen multiples nucleos, por lo que pueden haber muchos procesos eejcutandose de manera *paralela*

- Cuando hay mas procesos que nucleos, los 1ros se ponen en cola esperando su turno para ser ejecutados por un nucleo disponible. El OS se encarga de ejecutar procesos y pausarlos para ejecutar otros, y dar una sensacion de que todo se esta ejecutando de manera simultanea

- Los *threads* pueden hacer que varios nucleos ejecuten **distintos flujos de instrucciones** de un **mismo proceso** (nuestro programa, por ejemplo) al mismo tiempo

## Threads

- Thread: Unidad de ejecucion dentro de un proceso. LLeva la cuente de en que parte del codigo estamos ejecutando

- Lleva sus propias variables locales y en que parte del codigo de encuentra 

![Alt text](imgs/5_threads.png)

La ejecucion de varios threads sigue esta secuencia (*thread scheduling*) 
1. Se escoge un *thread* entre los que están disponible para ejecutar.
2. Se ejecuta un cierto número de instrucciones de ese *thread*, o durante un cierto tiempo.
3. Se deja el *thread* actual en espera.
4. Se vuelve al paso 1.

- En el caso de un solo nucleo, no hay paralelismo, pero la ejecucion es tan rapida entre threads que da la sensacion de una ejecucion en paralelo

- Esto hace que los programas sean mas lentos, pero a cambio es mas sencillo en el aspecto de la programacion, ya que solo nos preocupamos de que se ejecuten los threads, no de como estos saltan de uno a otro (manejado por el OS)

- En el caso de multiples nucleos: Se aprovecha el paralelismo ejecutando threads distintos en los varios nucleos. Dando como resultado una ejecucion del programa mas rapida

## Uso

1. **Separacion de funciones independientes**: Pueden trabajar sobre el mismo conjunto de datos, pero no se afectan entre si (pueden ejecutarse a la vez)

2. **Interfaces graficas**: Threads de computo pesado y threads de parte visual, por ejemplo

3. **Aplicaciones de multiples usuarios**

## Extra

- en print, el imprimir el texto y luego un `\n` son *dos intrucciones distintas*, por lo que es posible que el OS detenga un thread justo en medio de esto y se impriman ambos por separado

### Threads donde `target` tiene paremetros
- En el caso de funciones con parametros, podemos entregar argumentos usando los atributos `args` y `kwargs` del objeto thread

### Threads con OOP
- Podemos crear una clase que herede de un thread
- Para las instrucciones que ejecute la instancia del thread, se hace *override* al metodo `run()` (el que se ejecuta luego de `start()` en el thread).

## `join()`

- A veces es el programa principal (el `MainThread`) necesita que otros threads terminen antes de continuar ejecutandose
- `thread1.join(timeout=None)` detiene al programa principal hasta que `thread1` termine de ejecutarse
- Podemos especificar un `timeout` (en *s*) para esperar a `thread1` un tiempo determinado y luego seguir ejecutando el `MainThread`

- Cualquier thread puede **esperar** a otro usando el metodo `join()`

## `is_alive()`

- booleano para saber el estado del thread despues de un uso de join(), **despues de haber definido su tiempo maximo de espera**

## *Daemons*

- threads que no impiden que el programa termine aun que sigan corriendo
- podemos dejarlos corriendo y no preocuarnos de verificar su termino si el programa principal necesita terminar

- El programa principal termina cuando todos los threads no deamon terminan
- Un thread se especifica como daemon, se especifica en el parametro `daemon=True`
- No se puede cambiar el bool de daemon despues de haber iniciado el programa

- Utiles para ejecutar tareas no criticas para el funcionamoento del programa
- Para tareas esporadicas, periodicas (data save/load) o que se necesita que duren mientras el programa este activo (monitoreo de un recurso o variable)

## *Timers*

- subclase de `Thread` que ejecuta un proceso despues que ha pasado un cierto tiempo
- `cancel()`: permite cancelar la ejecucion del *timer* si se necesita
- `th.Timer(t, func_a_ejecutar, args, kwargs)` donde `t` es el tiempo en segundos despues el cual debe ejecutarse el thread

# Concurrencia

- Uso de threads psudo paralelos

- Threads que modifan un valor u atributo a la vez, generan problemas, ya que es posible que uno de los threads lea un valor no actualizado del atributo

1. Operacion **atomica**: operacion que un *thread* solo puede iniciar si ningun otro thread la esta ejecutando
- Instrucciones atomicas: Seccion critica

## Sincronizacion de *Threads*

### `Lock()`

- objeto que bloquea ciertas partes de un codigo (secciones criticas) para asegurar que un solo thread la esta ejecutando a la vez
- ``lock().acquire`` - bloquear
- ``lock().release`` - desbloquar

- La instancia de lock debe ser **compartido entre los threads** que necesitan acceder a la misma instruccion. Esto con el objetivo de que solo haya un thread que pueda pasar por encima del metodo `acquire()`
- utilizar como atributo de clase en clases que hereden de thread

### Locks y Context Managers (sentencia `with`)
- el `with` se encarga de llamar a `acquire()` al iniciar y `release()` al finalizar. Lo que encapsula a toda la seccion critica dentro de la identacion

### `Event` - señales entre *threads*

- util para unos threads esperen a otros
- Un `Event()` puede ser manipulado de las siguentes formas
1. `wait()` . El thread pausa su ejecucion y se queda esperando a que otro thread active la señal

2. `set()` Se activa la señal, todos los threads que la estaban esperando pueden continuar

3. `is_set()` un thread puede consultar si una señal esta activada sin necesidad de llamar a `wait()` y pausarse. Esto por ejemplo serviria para programar que hacen en caso de que la señal aun no esta activada

4. `clear()` apaga una señal para poder ser usada de nuevo

### *Deadlocks*

- Situacion donde **dos o mas threads se esperan mutuamente**

- Caso en con `Lock()`: Surgen cuando un thread bloquea un lock1 y otro un lock2, pero luego el primero necesita el lock2 para desbloquear lock1, y el segundo necesita el lock1 para desbloquear lock2, pero como lock2 y lock1 estan bloqueados, no se puede hacer nada 

- Caso con `Event()`: Surgen cuando se invierte el orden en el que se revisan y levantan señales. Cuando se **espera** una señal (que la active otro thread), **antes** de haber avisado que hizo su trabajo